In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [20]:
# Configuración visualizaciones
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Cargar datos
df = pd.read_csv("final_dataset_with_odds.csv")
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

Filas: 6000, Columnas: 55


In [35]:
# Primeras filas
display(df.head(5))

,Unnamed: 0,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,...,B365A,IWH,IWD,IWA,LBH,LBD,LBA,WHH,WHD,WHA
0,0,2002-08-17,Blackburn,Sunderland,0,0,NH,0,0,0,...,4.333,1.8,3.1,3.8,1.615,3.25,5.00,1.66,3.3,4.50
1,1,2002-08-17,Charlton,Chelsea,2,3,NH,0,0,0,...,2.200,2.9,3.0,2.2,2.800,3.20,2.20,2.75,3.1,2.30
2,2,2002-08-17,Everton,Tottenham,2,2,NH,0,0,0,...,2.750,2.3,3.0,2.7,2.250,3.20,2.75,2.30,3.1,2.75
3,3,2002-08-17,Fulham,Bolton,4,1,H,0,0,0,...,4.333,1.8,3.1,3.8,1.833,3.20,3.75,1.72,3.2,4.33
4,4,2002-08-17,Leeds,Man City,3,0,H,0,0,0,...,4.500,1.7,3.2,4.2,1.615,3.50,4.50,1.66,3.3,4.50
